In [ ]:
# ==========================================
# 📘 PHASE 4: Model Evaluation & Visualization
# ==========================================

import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report, confusion_matrix,
    balanced_accuracy_score, accuracy_score
)
from tensorflow.keras.models import load_model

# --------------------------------------
# 1️⃣ Mount Drive and Set Paths
# --------------------------------------
from google.colab import drive
drive.mount('/content/drive')

base_path = "/content/drive/MyDrive"
model_path = os.path.join(base_path, "AML2_Project_Models/cnn_model.h5")
data_path  = os.path.join(base_path, "AML2_Project_Preprocessed")

# Load model and test data
model = load_model(model_path)
X_test = np.load(os.path.join(data_path, "X_test.npy"))
y_test = np.load(os.path.join(data_path, "y_test.npy"))

class_names = ['NEV', 'ACK', 'SEK', 'BCC', 'MEL', 'SCC']

print("✅ Model & data loaded successfully!")
print(f"Test data shape: {X_test.shape}, Labels: {y_test.shape}")

# --------------------------------------
# 2️⃣ Load Training History (optional)
# --------------------------------------
import pickle
history_path = os.path.join(base_path, "AML2_Project_Models/training_history.pkl")

if os.path.exists(history_path):
    with open(history_path, 'rb') as f:
        history = pickle.load(f)
    print("✅ Training history loaded!")
else:
    history = None
    print("⚠️ History file not found. Skipping training curves.")

# --------------------------------------
# 3️⃣ Plot Training Curves (if available)
# --------------------------------------
if history:
    plt.figure(figsize=(12,5))

    # Accuracy
    plt.subplot(1,2,1)
    plt.plot(history['accuracy'], label='Train Accuracy')
    plt.plot(history['val_accuracy'], label='Val Accuracy')
    plt.title("Model Accuracy")
    plt.xlabel("Epochs"); plt.ylabel("Accuracy")
    plt.legend()

    # Loss
    plt.subplot(1,2,2)
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title("Model Loss")
    plt.xlabel("Epochs"); plt.ylabel("Loss")
    plt.legend()

    plt.show()

# --------------------------------------
# 4️⃣ Evaluate on Test Set
# --------------------------------------
print("\n🔍 Evaluating model on test data...")
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# FIX: Handle label format automatically
if len(y_test.shape) == 1:
    y_true = y_test
else:
    y_true = np.argmax(y_test, axis=1)

# Compute metrics
acc = accuracy_score(y_true, y_pred)
bal_acc = balanced_accuracy_score(y_true, y_pred)

print(f"\n✅ Standard Accuracy: {acc*100:.2f}%")
print(f"✅ Balanced Accuracy: {bal_acc*100:.2f}%\n")

print("📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# --------------------------------------
# 5️⃣ Confusion Matrix
# --------------------------------------
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# --------------------------------------
# 6️⃣ Visualise Random Predictions
# --------------------------------------
import random

def plot_predictions(X, y_true, y_pred, class_names, n=9):
    idxs = random.sample(range(len(X)), n)
    plt.figure(figsize=(12,12))
    for i, idx in enumerate(idxs):
        plt.subplot(3,3,i+1)
        plt.imshow(X[idx])
        true_label = class_names[y_true[idx]]
        pred_label = class_names[y_pred[idx]]
        color = "green" if true_label == pred_label else "red"
        plt.title(f"True: {true_label}\nPred: {pred_label}", color=color)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

plot_predictions(X_test, y_true, y_pred, class_names)

# --------------------------------------
# 7️⃣ Accuracy Comparison Bar Plot
# --------------------------------------
plt.figure(figsize=(6,4))
plt.bar(["Standard Accuracy", "Balanced Accuracy"], [acc*100, bal_acc*100],
        color=["skyblue", "orange"])
plt.title("Accuracy Comparison")
plt.ylabel("Percentage (%)")
for i, v in enumerate([acc*100, bal_acc*100]):
    plt.text(i, v + 1, f"{v:.2f}%", ha='center', fontweight='bold')
plt.ylim(0, 100)
plt.show()

# --------------------------------------
# 8️⃣ Save Evaluation Results
# --------------------------------------
results_dir = os.path.join(base_path, "AML2_Project_Results")
os.makedirs(results_dir, exist_ok=True)

np.save(os.path.join(results_dir, "y_true.npy"), y_true)
np.save(os.path.join(results_dir, "y_pred.npy"), y_pred)

print(f"\n✅ Evaluation results saved to: {results_dir}")


In [ ]:

# ======================================================
# 📘 PHASE 5: Explainable AI – LIME + SHAP Visualization
# ======================================================

import os
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.keras.models import load_model

# --------------------------------------------
# 1️⃣ Mount Drive and Load Model + Data
# --------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

base_path = "/content/drive/MyDrive"
model_path = os.path.join(base_path, "AML2_Project_Models/cnn_model.h5")
data_path  = os.path.join(base_path, "AML2_Project_Preprocessed")

model = load_model(model_path)
X_test = np.load(os.path.join(data_path, "X_test.npy"))
y_test = np.load(os.path.join(data_path, "y_test.npy"))

class_names = ['NEV', 'ACK', 'SEK', 'BCC', 'MEL', 'SCC']

# Ensure y_test is integer encoded
if len(y_test.shape) == 1:
    y_true = y_test
else:
    y_true = np.argmax(y_test, axis=1)

print("✅ Model and data loaded successfully.")
print(f"Test shape: {X_test.shape}, Labels: {y_true.shape}")

# --------------------------------------------
# 2️⃣ Choose Samples to Explain
# --------------------------------------------
# Randomly pick 5 test images
sample_idxs = random.sample(range(len(X_test)), 5)
X_samples = X_test[sample_idxs]
y_samples = y_true[sample_idxs]

# Model predictions
pred_probs = model.predict(X_samples)
y_preds = np.argmax(pred_probs, axis=1)

# --------------------------------------------
# 3️⃣ Local Explanation with LIME
# --------------------------------------------
!pip install lime --quiet
from lime import lime_image
from skimage.segmentation import mark_boundaries

explainer = lime_image.LimeImageExplainer()

for i, idx in enumerate(sample_idxs):
    explanation = explainer.explain_instance(
        X_test[idx].astype('double'),
        model.predict,
        top_labels=1,
        hide_color=0,
        num_samples=1000
    )

    temp, mask = explanation.get_image_and_mask(
        label=y_preds[i],
        positive_only=False,
        hide_rest=False,
        num_features=5,
        min_weight=0.0
    )

    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.imshow(X_test[idx])
    plt.title(f"Original\nTrue: {class_names[y_true[idx]]}\nPred: {class_names[y_preds[i]]}")

    plt.subplot(1,2,2)
    plt.imshow(mark_boundaries(temp/255.0, mask))
    plt.title("LIME Explanation")
    plt.axis('off')
    plt.show()

# --------------------------------------------
# 4️⃣ Global Explanation with SHAP
# --------------------------------------------
!pip install shap --quiet
import shap

# Compute SHAP values for a subset of test images
subset_idx = np.random.choice(len(X_test), 50, replace=False)
X_subset = X_test[subset_idx]

# Select 20 background samples for reference
background = X_subset[:20]

# Use GradientExplainer for CNN models
explainer = shap.GradientExplainer(model, background)

# Calculate SHAP values for 10 test samples
shap_values = explainer.shap_values(X_subset[20:30])

# Normalise SHAP values for better visual contrast
shap_values = [sv / (np.max(np.abs(sv)) + 1e-8) for sv in shap_values]

# Plot global SHAP image summary
plt.title("SHAP Summary – Global Feature Influence (Normalised)")
shap.image_plot(shap_values, X_subset[20:30])

# --------------------------------------------
# 5️⃣ Save Results
# --------------------------------------------
results_dir = os.path.join(base_path, "AML2_Project_Results")
os.makedirs(results_dir, exist_ok=True)
np.save(os.path.join(results_dir, "shap_values.npy"), shap_values)

print(f"✅ Explanations saved to: {results_dir}")


In [ ]:
import json
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# --------------------------------------------
# Phase 6 – Model Summary & Report
# --------------------------------------------

# Evaluate model again to refresh metrics
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# If y_test is already integer labels, skip argmax
try:
    y_true = np.argmax(y_test, axis=1)
except:
    y_true = y_test

# Classification report and confusion matrix
report = classification_report(y_true, y_pred, target_names=[f'Class {i}' for i in range(len(np.unique(y_true)))], output_dict=True)
conf_mat = confusion_matrix(y_true, y_pred)

# Print and save results
print("✅ Model Evaluation Summary\n")
print(json.dumps(report, indent=4))
print("\nConfusion Matrix:\n", conf_mat)

# Plot confusion matrix
plt.figure(figsize=(7,6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix – Skin Lesion Classification")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()
